# Model Selection and Hyperparameter Tuning in Spark Scala

# 1.- Cross Validation

After identifying the best ParamMap, CrossValidator finally re-fits the Estimator using the best ParamMap and the entire dataset.

In [1]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.sql.Row

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-FQ2BOOJ:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1557917998169)
SparkSession available as 'spark'


import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.sql.Row


## 1.1.- Synthetic toy data - training set

In [2]:
// Prepare training data from a list of (id, text, label) tuples.
val training = spark.createDataFrame(Seq(
  (0L, "a b c d e spark", 1.0),
  (1L, "b d", 0.0),
  (2L, "spark f g h", 1.0),
  (3L, "hadoop mapreduce", 0.0),
  (4L, "b spark who", 1.0),
  (5L, "g d a y", 0.0),
  (6L, "spark fly", 1.0),
  (7L, "was mapreduce", 0.0),
  (8L, "e spark program", 1.0),
  (9L, "a e c l", 0.0),
  (10L, "spark compile", 1.0),
  (11L, "hadoop software", 0.0)
)).toDF("id", "text", "label")

training: org.apache.spark.sql.DataFrame = [id: bigint, text: string ... 1 more field]


In [3]:
training.show

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
|  4|     b spark who|  1.0|
|  5|         g d a y|  0.0|
|  6|       spark fly|  1.0|
|  7|   was mapreduce|  0.0|
|  8| e spark program|  1.0|
|  9|         a e c l|  0.0|
| 10|   spark compile|  1.0|
| 11| hadoop software|  0.0|
+---+----------------+-----+



## 1.2.- Pipeline setup

In [4]:
// Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
val tokenizer = new Tokenizer()
  .setInputCol("text")
  .setOutputCol("words")
val hashingTF = new HashingTF()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("features")
val lr = new LogisticRegression()
  .setMaxIter(10)
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))

tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_925d1345009f
hashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_ad3a27a2a48b
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_8baceda80076
pipeline: org.apache.spark.ml.Pipeline = pipeline_950fc1751bcf


## 1.3.- Parameter grid

In [5]:
// We use a ParamGridBuilder to construct a grid of parameters to search over.
// With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
// this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
val paramGrid = new ParamGridBuilder()
  .addGrid(hashingTF.numFeatures, Array(10, 100, 1000))
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	hashingTF_ad3a27a2a48b-numFeatures: 10,
	logreg_8baceda80076-regParam: 0.1
}, {
	hashingTF_ad3a27a2a48b-numFeatures: 10,
	logreg_8baceda80076-regParam: 0.01
}, {
	hashingTF_ad3a27a2a48b-numFeatures: 100,
	logreg_8baceda80076-regParam: 0.1
}, {
	hashingTF_ad3a27a2a48b-numFeatures: 100,
	logreg_8baceda80076-regParam: 0.01
}, {
	hashingTF_ad3a27a2a48b-numFeatures: 1000,
	logreg_8baceda80076-regParam: 0.1
}, {
	hashingTF_ad3a27a2a48b-numFeatures: 1000,
	logreg_8baceda80076-regParam: 0.01
})


## 1.4.- Cross-validator setup

In [6]:
// We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
// This will allow us to jointly choose parameters for all Pipeline stages.
// A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
// Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
// is areaUnderROC.
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(2)  // Use 3+ in practice
  .setParallelism(2)  // Evaluate up to 2 parameter settings in parallel


cv: org.apache.spark.ml.tuning.CrossValidator = cv_bfcf8888a3ab


With **BinaryClassificationEvaluator** the available metrics are (see https://spark.apache.org/docs/2.1.0/api/scala/index.html#org.apache.spark.mllib.evaluation.BinaryClassificationMetrics):

* areaUnderROC (default
* areaUnderPR
* fMeasureByThreshold
* precisionByThreshold
* recallByThreshold
* roc

In [7]:
// Run cross-validation, and choose the best set of parameters.
val cvModel = cv.fit(training)

2019-05-15 12:02:42 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-05-15 12:02:42 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_bfcf8888a3ab


In [8]:
// average metrics (ROC) for each point in the grid
cvModel.avgMetrics

res1: Array[Double] = Array(0.625, 0.6125, 0.875, 0.8875, 0.85, 0.85)


## 1.5.- Synthetic toy data - test set

In [10]:
// Prepare test documents, which are unlabeled (id, text) tuples.
val test = spark.createDataFrame(Seq(
  (4L, "spark i j k"),`
  (5L, "l m n"),
  (6L, "mapreduce spark"),
  (7L, "apache hadoop")
)).toDF("id", "text")

test: org.apache.spark.sql.DataFrame = [id: bigint, text: string]


In [11]:
test.show

+---+---------------+
| id|           text|
+---+---------------+
|  4|    spark i j k|
|  5|          l m n|
|  6|mapreduce spark|
|  7|  apache hadoop|
+---+---------------+



## 1.6.- Prediction on test data

In [12]:
// Make predictions on test documents. cvModel uses the best model found (lrModel).
cvModel.transform(test)
  .select("id", "text", "probability", "prediction")
  .collect()
  .foreach { case Row(id: Long, text: String, prob: Vector, prediction: Double) =>
    println(s"($id, $text) --> prob=$prob, prediction=$prediction")
  }

(4, spark i j k) --> prob=[0.12566260711357224,0.8743373928864279], prediction=1.0
(5, l m n) --> prob=[0.995215441016286,0.004784558983714], prediction=0.0
(6, mapreduce spark) --> prob=[0.30696895232625965,0.6930310476737404], prediction=1.0
(7, apache hadoop) --> prob=[0.8040279442401378,0.19597205575986223], prediction=0.0


# 2.- Train - Validation Split

In [14]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}


## 2.1.- Load data, train/test split

In [16]:
// Prepare training and test data.
val data = spark.read.format("libsvm").load("/home/jmalbornoz/Downloads/spark-2.4.0-bin-hadoop2.7/data/mllib/sample_linear_regression_data.txt")
val Array(training, test) = data.randomSplit(Array(0.9, 0.1), seed = 12345)

2019-05-15 15:15:02 WARN  LibSVMFileFormat:66 - 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


data: org.apache.spark.sql.DataFrame = [label: double, features: vector]
training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


## 2.2.- Linear regression estimator

In [17]:
val lr = new LinearRegression().setMaxIter(10)

lr: org.apache.spark.ml.regression.LinearRegression = linReg_54f5c5a30375


## 2.3.- Parameter grid

In [18]:
// We use a ParamGridBuilder to construct a grid of parameters to search over.
// TrainValidationSplit will try all combinations of values and determine best model using
// the evaluator.
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .addGrid(lr.fitIntercept)
  .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_54f5c5a30375-elasticNetParam: 0.0,
	linReg_54f5c5a30375-fitIntercept: true,
	linReg_54f5c5a30375-regParam: 0.1
}, {
	linReg_54f5c5a30375-elasticNetParam: 0.0,
	linReg_54f5c5a30375-fitIntercept: false,
	linReg_54f5c5a30375-regParam: 0.1
}, {
	linReg_54f5c5a30375-elasticNetParam: 0.5,
	linReg_54f5c5a30375-fitIntercept: true,
	linReg_54f5c5a30375-regParam: 0.1
}, {
	linReg_54f5c5a30375-elasticNetParam: 0.5,
	linReg_54f5c5a30375-fitIntercept: false,
	linReg_54f5c5a30375-regParam: 0.1
}, {
	linReg_54f5c5a30375-elasticNetParam: 1.0,
	linReg_54f5c5a30375-fitIntercept: true,
	linReg_54f5c5a30375-regParam: 0.1
}, {
	linReg_54f5c5a30375-elasticNetParam: 1.0,
	linReg_54f5c5a30375-fitIntercept: false,
	linReg_54f5c5a30375-regPar...

## 2.4.- Train/validation setup

In [19]:
// In this case the estimator is simply the linear regression.
// A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(lr)
  .setEvaluator(new RegressionEvaluator)
  .setEstimatorParamMaps(paramGrid)
  // 80% of the data will be used for training and the remaining 20% for validation.
  .setTrainRatio(0.8)
  // Evaluate up to 2 parameter settings in parallel
  .setParallelism(2)

trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_81ab3cac8664


In [20]:
// Run train validation split, and choose the best set of parameters.
val model = trainValidationSplit.fit(training)

2019-05-15 15:21:14 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
2019-05-15 15:21:14 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_81ab3cac8664


## 2.5.- Prediction on test data

In [23]:
// Make predictions on test data. model is the model with combination of parameters
// that performed best.
model.transform(test)
  .select("features", "label", "prediction")
  .show()

+--------------------+--------------------+--------------------+
|            features|               label|          prediction|
+--------------------+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -23.51088409032297| -1.6659388625179559|
|(10,[0,1,2,3,4,5,...| -21.432387764165806|  0.3400877302576284|
|(10,[0,1,2,3,4,5,...| -12.977848725392104|-0.02335359093652395|
|(10,[0,1,2,3,4,5,...| -11.827072996392571|  2.5642684021108417|
|(10,[0,1,2,3,4,5,...| -10.945919657782932| -0.1631314487734783|
|(10,[0,1,2,3,4,5,...|  -10.58331129986813|   2.517790654691453|
|(10,[0,1,2,3,4,5,...| -10.288657252388708| -0.9443474180536754|
|(10,[0,1,2,3,4,5,...|  -8.822357870425154|  0.6872889429113783|
|(10,[0,1,2,3,4,5,...|  -8.772667465932606|  -1.485408580416465|
|(10,[0,1,2,3,4,5,...|  -8.605713514762092|   1.110272909026478|
|(10,[0,1,2,3,4,5,...|  -6.544633229269576|  3.0454559778611285|
|(10,[0,1,2,3,4,5,...|  -5.055293333055445|  0.6441174575094268|
|(10,[0,1,2,3,4,5,...|  -